In [1]:
%load_ext autoreload
%autoreload 2



In [13]:
from runner import BENCHMARK_APPLICATIONS
from process_results import *
import os
import pandas as pd

In [21]:

data = {
    "name": [],
    "num_exce": [],
    "num_frame": [],
    "is_async": [],
    "logged": [],
    "dynamic": [],
    "static": [],
    "hybrid_online": [],
    "hybrid_offline": [],
    "distance": []
}

for name, cls in BENCHMARK_APPLICATIONS.items():
    data["name"].append(name)
    print(name)
    app = cls()
    static_result_path = app.get_latest_result("static")
    exceptions = read_exceptions(os.path.join(static_result_path, "exception.json"))
    ground_truth = app.read_ground_truth()
    dynamic_dependency = app.read_latest_dynamic_dependency()
    root_cause_in_log = check_root_cause_in_log(ground_truth, app.origin_log_path)
    data["is_async"].append(app.is_async)
    data["logged"].append(root_cause_in_log)

    (tp, fp, fn) = process_dependency_result(dynamic_dependency, ground_truth)
    data["dynamic"].append(f"{tp}/{fp}/{fn}")

    static_dependency = app.read_latest_static_dependency()
    (tp, fp, fn) = process_dependency_result(static_dependency, ground_truth)
    data["static"].append(f"{tp}/{fp}/{fn}")

    hybrid_online, hybrid_offline = app.read_latest_hybrid_dependency()
    (tp, fp, fn) = process_dependency_result(hybrid_online, ground_truth)
    data["hybrid_online"].append(f"{tp}/{fp}/{fn}")
    (tp, fp, fn) = process_dependency_result(hybrid_offline, ground_truth)
    data["hybrid_offline"].append(f"{tp}/{fp}/{fn}")
    data["distance"].append(get_exception_distance(ground_truth,
                                                   os.path.join(app.get_latest_result("dynamic"), "exception.json")))


    data['num_exce'].append(len(exceptions))
    data["num_frame"].append(len(open(os.path.join(app.get_latest_result(
        "dynamic"), "affected-var-results.json")).readlines()))

df = pd.DataFrame(data)
print(df)

jena_324
fineract_1211
mapreduce_6654
nifi_8249
hadoop_17812
wicket_6249
hdfs_4128
hive_13410
             name  num_exce  num_frame  is_async  logged dynamic static  \
0        jena_324      1396       5958      True    True   1/0/0  0/1/1   
1   fineract_1211        66        305     False    True   1/0/0  1/0/0   
2  mapreduce_6654        58        486     False    True   1/0/0  0/0/1   
3       nifi_8249        79       2642      True    True   1/0/0  0/2/1   
4    hadoop_17812        33        127     False   False   1/0/0  0/1/1   
5     wicket_6249        12        103      True   False   1/0/0  1/0/0   
6       hdfs_4128       124        473      True   False   1/0/0  0/0/1   
7      hive_13410        53        278      True   False   4/0/0  0/0/4   

  hybrid_online hybrid_offline  distance  
0         1/0/0          0/0/1       986  
1         0/0/1          0/0/1         0  
2         1/0/0          0/0/1        10  
3         0/0/1          0/2/1         0  
4         1/0/0